In [ ]:
import pandas as pd 
import numpy as np 
import ast

In [3]:
df = pd.read_csv('/Users/kelvinfoo/Desktop/AI Masters/AirBnb Project/Data/listing_with_cluster.csv')

# Convert amenities and neighbourhood to lower case 
df['amenities'] = df['amenities'].apply(ast.literal_eval)
df['amenities'] = df['amenities'].apply(lambda x: [item.lower() for item in x])
df['neighbourhood_cleansed'] = df['neighbourhood_cleansed'].str.lower()

# Convert Boolean attribute to True and False 
df['host_is_superhost'] = df['host_is_superhost'].map({'t': True, 'f': False})
df['host_identity_verified'] = df['host_identity_verified'].map({'t': True, 'f': False})

df['embedding'] = df['embedding'].apply(lambda x: np.array([float(i) for i in x.strip("[]").split()]) if isinstance(x, str) else x)
print(f"Data shape: {df.shape}")
df.head()


Data shape: (24977, 17)


,Unnamed: 0.1,Unnamed: 0,name,description,neighborhood_overview,host_is_superhost,host_identity_verified,neighbourhood_cleansed,room_type,accommodates,bathrooms,bedrooms,amenities,price,review_scores_value,embedding,agg_cluster
0,3,3,Comfy bedroom near River pier & BTS Taksin.,This is one of our 'Escape' sleep room : hoste...,"Charoen krung street, River pier, Iconsiam, Su...",True,True,sathon,Hotel room,2,1,1,"[shampoo, hangers, breakfast, hot water, tv wi...",1536,4.69,"[0.060480874, 0.0582088716, 0.0019648266, 0.05...",2
1,4,4,Central Bangkok 3 Bedroom Apartment,We are Airbnb Super Hosts. Book our apartment ...,Sukhumvit Rd is the shopping and restaurant di...,False,True,khlong toei,Entire home/apt,5,3,3,"[shampoo, tv with standard cable, extra pillow...",5601,4.76,"[0.0409337059, -0.0361530222, 0.0519887842, 0....",0
2,10,10,Oasis in the heart of Bangkok,In the mid of vibrant in the center of Bangkok...,Local small food market during the day close t...,False,True,ratchathewi,Entire home/apt,2,1,1,"[shampoo, freezer, kitchen, shared pool, dedic...",1366,4.86,"[0.0329679251, 0.0403496213, -0.0546748601, 0....",0
3,12,12,Monthly rent 2Beds/2Baths quiet APT at BTS,This is a monthly rented apartment. Our clean ...,Taksin is a very local residential neighborhoo...,True,True,khlong san,Entire home/apt,4,2,2,"[shampoo, hangers, kitchen, free parking on pr...",1810,4.71,"[-0.0121601848, 0.019530328, -0.015967356, 0.0...",0
4,13,13,Nice room with superb city view,Our cool and comfortable one bedroom apartment...,It is very center of Bangkok and easy access t...,False,True,ratchathewi,Entire home/apt,2,1,1,"[shampoo, kitchen, hangers, elevator, smoke al...",1878,4.75,"[0.111119725, -0.0271609426, 0.037512105, 0.00...",2


### **Building Recommender System**

In [18]:
def get_user_preference(): 
    try: 
        def positive_int_input(prompt): 
            while True: 
                value = input(prompt)
                if not value: 
                    print("This field is required. Please enter a number greater than or equal to 1.")
                else: 
                    try: 
                        value = int(value)
                        if value >= 1: 
                            return value 
                        else: 
                            print("Please enter a number greater than or equal to 1.")
                    except ValueError: 
                        print("Invalid input. Please enter a valid number greater than or equal to 1.")

        def greater_than_zero_input(prompt): 
            while True: 
                value = input(prompt)
                if not value: 
                    print("This field is required. Please enter a number greater than or equal to 0.")
                else: 
                    try: 
                        value = int(value) 
                        if value > 0: 
                            return value 
                        else: 
                            print("Please enter a number greater than or equal to 0.")
                    except ValueError: 
                        print("Invalid input. Please enter a valid number greater than or equal to 0.")
        
        accommodates = positive_int_input("Minimum number of guest(s) (must be 1 or more): ")
        price = greater_than_zero_input("Maximum price: ")
        bathrooms = greater_than_zero_input("Minimum number of bathroom(s): ")
        bedrooms = greater_than_zero_input("Minimum number of bedroom(s): ")

        verifiedHost = input("Only verified host (Y/N): ").strip().upper()
        if verifiedHost == "YES":
            verifiedHost = "Y"
        elif verifiedHost == "NO":
            verifiedHost = "N"
        verifiedHost = True if verifiedHost == "Y" else False if verifiedHost == "N" else None
        superHost = input("Only super host (Y/N): ").strip().upper()
        if superHost == "YES":
            superHost = "Y"
        elif superHost == "NO":
            superHost = "N"
        superHost = True if superHost == "Y" else False if superHost == "N" else None
        amenities = input("Preferred amenities, seperated by commas: ")
        amenities = [a.strip() for a in amenities.strip(",")] if amenities else None

        listing_name = input("I would prefer similar listings as: ")
        listing_name = listing_name if listing_name else None 

        userPreferences = {"accommodates": accommodates,
                       "price": price,
                       "bathrooms": bathrooms,
                       "bedrooms": bedrooms,
                       "verified_host": verifiedHost,
                       "super_host": superHost,
                       "amenities": amenities, 
                       "listing_name": listing_name}
        return userPreferences
    
    except ValueError: 
        print("Unexpected input error. Please try again.")
        return get_user_preference()

In [19]:
def filter_listing(userPreferences):
    preferred_listing_cluster = df[df['name'] == userPreferences['listing_name']]['agg_cluster'].iloc[0] if userPreferences['listing_name'] is not None else True
    cluster_filter = df['agg_cluster'] == preferred_listing_cluster 

    accommodatesFilter = df['accommodates'] == userPreferences['accommodates'] if userPreferences['accommodates'] is not None else True
    priceFilter = df['price'] <= userPreferences['price'] if userPreferences['price'] is not None else True
    bathroomsFilter = df['bathrooms'] >= userPreferences['bathrooms'] if userPreferences['bathrooms'] is not None else True
    bedroomsFilter = df['bedrooms'] >= userPreferences['bedrooms'] if userPreferences['bedrooms'] is not None else True
    
    verified_hostFilter = (df['host_identity_verified'] == True) if userPreferences['verified_host'] == True else True
    super_hostFilter = (df['host_is_superhost'] == True) if userPreferences['super_host'] == True else True

    tailoredListings = df[accommodatesFilter &
                          priceFilter &
                          bathroomsFilter &
                          bedroomsFilter &
                          verified_hostFilter &
                          super_hostFilter & 
                          cluster_filter]

    if userPreferences['amenities']: 
        tailoredListings = tailoredListings[tailoredListings['amenities'].apply(lambda am: any(any(keyword in amen for amen in am) for keyword in userPreferences['amenities']))]
    return tailoredListings

In [20]:
def main(): 
    userPreferences = get_user_preference()
    user_filteredListings = filter_listing(userPreferences)

    final_recommendation = user_filteredListings.iloc[:, 2:14]
    final_recommendation = final_recommendation.rename(columns = {'host_is_superhost': 'superhost', 'host_identity_verified': 'verified_host', 'neighborhood_cleansed': 'neighborhood', 'review_scores_value': 'review_score'})
    return final_recommendation

if __name__ == "__main__": 
    print("Here are your recommendations: ")
    final_recommendation = main()
    display(final_recommendation)

Here are your recommendations: 


,name,description,neighborhood_overview,superhost,verified_host,neighbourhood_cleansed,room_type,accommodates,bathrooms,bedrooms,amenities,price
201,S1 hostel (Dorm) Sathorn Bangkok,"Stylish budget hostel in the midst of Bangkok,...",Our hostel is located on small greenery road. ...,True,True,sathon,Shared room,1,1,1,"[shampoo, body soap, extra pillows and blanket...",380
357,"505 Cozy studio, BTS Phromphong","Welcome to our home, centrally-located townhou...",NaN,True,True,vadhana,Private room,1,1,1,"[shampoo, iron, piano, shower gel, wifi, clean...",937
631,Corner Room+ Pool+ Sky train,"The link 2, 5 minutes walk to Onnuch BTS Skytr...",NaN,False,True,vadhana,Private room,1,1,1,"[shared pool, dedicated workspace, elevator, a...",351
758,Stylish Single room near Tong lo Station Sukhu...,We offer the trendiest room in the heart of ci...,Thong Lo area is the upper scale residential a...,False,True,vadhana,Private room,1,1,1,"[tv with standard cable, wifi, air conditioning]",570
784,Warmy @ 600m MRT Bangson Station,-600m to MRT Bangson Station by walk<br />-10k...,My place location is not the ideal location fo...,True,True,bang sue,Entire home/apt,1,1,1,"[city skyline view, shampoo, iron, hot water k...",850
...,...,...,...,...,...,...,...,...,...,...,...,...
24585,Lower Bed in Female Only dorm 10 Beds,Located within less than 1 km of Bangkok Natio...,NaN,False,True,phra nakhon,Shared room,1,2,1,"[air conditioning, wifi]",696
24597,Upper Bed in Female Only dorm 10 Beds,Located within less than 1 km of Bangkok Natio...,NaN,False,True,phra nakhon,Shared room,1,2,1,"[air conditioning, wifi]",616
24598,Lower bed in Female Only dorm 4 Beds,Located within less than 1 km of Bangkok Natio...,NaN,False,True,phra nakhon,Shared room,1,2,1,"[air conditioning, wifi]",696
24599,Upper bed in Female Only dorm 4 Beds,Located within less than 1 km of Bangkok Natio...,NaN,False,True,phra nakhon,Shared room,1,2,1,"[air conditioning, wifi]",616
